In [11]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [31]:
connection = create_server_connection("127.0.0.1", "root", "")

MySQL Database connection successful


In [22]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

In [24]:
create_database_query = "CREATE DATABASE School"
create_database(connection, create_database_query)

Database created successfully


In [33]:
cursor = connection.cursor()
cursor.execute("USE meubanco;")

In [37]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [39]:
results = read_query(connection, """SELECT * FROM pessoas;""")

InternalError: Unread result found